#### The Battle of Neighborhoods

## Real Estate Development in Brooklyn New York

**The Problem:**

In order to invest in properties with the highest return on investment, a real estate developer is exploring neighborhoods in Brooklyn, NY to predict which neighborhoods are more promising in their future growth. They believe that neighborhoods with best opportunities are the ones that share similar charactersitics in terms of trending venues with the ones that have already had a fast growth.
The neighborhoods that have already grown in the past few years are as following:

* Dumbo
* Downtown Brookly
* Fort Greene


**The Solution:**

To find this similarity, my method would be to cluster all neighborhoods in Brooklyn and find which neighborhoods will be in the same cluster as the three above neighborhoods selected by the real estate team as the models.



**The Method:**

First of all, I need a dataset of all brooklyn neighborhoods. The link below provides a free access to this dataset on web that I will use: https://geo.nyu.edu/catalog/nyu_2451_34572

I will use the data of neighborhoods from the dataset above to capture latitude and longtitude of all neighborhoods in Brooklyn and then use the geodata to get all trending venues info from Foursquare.

## Import & Process Data

#### 1. Import New York neighborhoods data and filter Brookly

In [1]:
## Import all required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
## Add neighborhoods content to the table

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
## use geopy to get latitude and longtitude of NYC

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
## filter brooklyn neighborhoods

brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


#### 2. Visualize Brooklyn neighborhoods

In [10]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

#### 3. Link to Foursquare

In [11]:
## define foursquare credentials

CLIENT_ID = 'AECVBA3XRV42QNIDW4HRTA1YPWEINXTOE5FT5VD3E215X2UH' # your Foursquare ID
CLIENT_SECRET = '30ESAB4SZDGTFKE00BYBN5U0LS1L5MGX2HHMHHJGRFDLUGQM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AECVBA3XRV42QNIDW4HRTA1YPWEINXTOE5FT5VD3E215X2UH
CLIENT_SECRET:30ESAB4SZDGTFKE00BYBN5U0LS1L5MGX2HHMHHJGRFDLUGQM


In [14]:
## create a funcion to repeat the same process for all neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

brooklyn_venues.shape
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [16]:
## group and sort neighborhoods by number of venues

sortedb = brooklyn_venues.groupby('Neighborhood').count()

sortedb = sortedb.sort_values(by = ["Venue"], ascending = False)

sortedb.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Downtown,200,200,200,200,200,200
Carroll Gardens,200,200,200,200,200,200
South Side,200,200,200,200,200,200
Cobble Hill,200,200,200,200,200,200
Brooklyn Heights,200,200,200,200,200,200


## Analyze and Cluseter Neighborhoods

#### 1. Group the neighborhoods and find the 10 top most venues

In [17]:
## analyze the neighborhood

# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Adult Boutique,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Ve

In [18]:
## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

In [19]:
## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

In [20]:
## put this into pandas data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Liquor Store,Bubble Tea Shop,Sushi Restaurant,Italian Restaurant,Donut Shop,Fast Food Restaurant,Kids Store,Deli / Bodega
1,Bay Ridge,Spa,Italian Restaurant,Bagel Shop,Greek Restaurant,American Restaurant,Bar,Pizza Place,Thai Restaurant,Chinese Restaurant,Ice Cream Shop
2,Bedford Stuyvesant,Deli / Bodega,Pizza Place,Café,Bar,Coffee Shop,Gourmet Shop,BBQ Joint,New American Restaurant,Vietnamese Restaurant,Cocktail Bar
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Bakery,Sushi Restaurant,Ice Cream Shop,Pizza Place,Donut Shop,Bagel Shop,Cha Chaan Teng,Liquor Store
4,Bergen Beach,Harbor / Marina,Baseball Field,Donut Shop,Playground,Athletics & Sports,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


#### 2. Cluster Neighborhoods

In [22]:
# set number of clusters
kclusters = 10

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 4, 0, 6, 2, 2, 0])

In [23]:
## Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Spa,Italian Restaurant,Bagel Shop,Greek Restaurant,American Restaurant,Bar,Pizza Place,Thai Restaurant,Chinese Restaurant,Ice Cream Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Italian Restaurant,Bakery,Sushi Restaurant,Ice Cream Shop,Pizza Place,Donut Shop,Bagel Shop,Cha Chaan Teng,Liquor Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,2,Bank,Pizza Place,Bakery,Latin American Restaurant,Mexican Restaurant,Ice Cream Shop,Gym,Mobile Phone Shop,Pharmacy,Breakfast Spot
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Bar,Cocktail Bar,Coffee Shop,Pizza Place,Yoga Studio,French Restaurant,Record Shop,Mexican Restaurant,Boutique,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Italian Restaurant,Pizza Place,Martial Arts Dojo,Bus Station,Lounge,Bakery,Sporting Goods Shop,Diner,Pharmacy,Liquor Store


In [24]:
## visualize

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Find Results

Back to the original question, we need to find that which neighborhoods are in the same cluster as "Downtown", "Dumbo", and "Fort Greene". So first, we need to find the cluster of these three neighborhoods and then find out what other neighborhoods share the same cluster.

In [57]:
downtown = brooklyn_merged.loc[brooklyn_merged['Neighborhood'] == "Downtown"].reset_index().at[0, "Cluster Labels"]
dumbo = brooklyn_merged.loc[brooklyn_merged['Neighborhood'] == "Dumbo"].reset_index().at[0, "Cluster Labels"]
FortGreene = brooklyn_merged.loc[brooklyn_merged['Neighborhood'] == "Downtown"].reset_index().at[0, "Cluster Labels"]

print("The cluster for our target neighborhoods are: \n", "Downtown: ", str(downtown), "\n", "Dumbo: ", str(dumbo), "\n", "Fort Greene: ", str(FortGreene))

The cluster for our target neighborhoods are: 
 Downtown:  0 
 Dumbo:  0 
 Fort Greene:  0


The result show that all three neighborhoods are in the same **Cluster 0**, so for the last step we need to find all other neighborhoods that are in the cluster 0.

In [58]:
resutl = brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Bagel Shop,Greek Restaurant,American Restaurant,Bar,Pizza Place,Thai Restaurant,Chinese Restaurant,Ice Cream Shop
3,Greenpoint,Bar,Cocktail Bar,Coffee Shop,Pizza Place,Yoga Studio,French Restaurant,Record Shop,Mexican Restaurant,Boutique,Café
9,Crown Heights,Pizza Place,Museum,Café,Bookstore,Bagel Shop,Farmers Market,Burger Joint,Deli / Bodega,Playground,Salon / Barbershop
12,Windsor Terrace,Diner,Deli / Bodega,Plaza,Park,Café,Grocery Store,Sushi Restaurant,Beer Store,Bookstore,Coffee Shop
13,Prospect Heights,Bar,Mexican Restaurant,Cocktail Bar,Wine Shop,Bakery,Ice Cream Shop,Gourmet Shop,Thai Restaurant,Yoga Studio,Pizza Place
15,Williamsburg,Bagel Shop,Coffee Shop,Pizza Place,Bar,Steakhouse,Korean Restaurant,Taco Place,Latin American Restaurant,Clothing Store,Yoga Studio
16,Bushwick,Bar,Deli / Bodega,Mexican Restaurant,Coffee Shop,Pizza Place,Discount Store,Bakery,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Café
17,Bedford Stuyvesant,Deli / Bodega,Pizza Place,Café,Bar,Coffee Shop,Gourmet Shop,BBQ Joint,New American Restaurant,Vietnamese Restaurant,Cocktail Bar
18,Brooklyn Heights,Yoga Studio,Park,Deli / Bodega,Gym,Pizza Place,Italian Restaurant,Cosmetics Shop,Plaza,Pet Store,Ice Cream Shop
19,Cobble Hill,Bar,Playground,Pizza Place,Italian Restaurant,Coffee Shop,Yoga Studio,Ice Cream Shop,Cocktail Bar,Deli / Bodega,Bakery


In [67]:
result = brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0]["Neighborhood"].tolist()

In [75]:
print("There are ", str(len(result)-3), "following neighborhoods in brookly sharing similar characteristics as our target neighborhoods.")
print("----------------------")
for neighborhood in result:
    print(neighborhood)

There are  47 following neighborhoods in brookly sharing similar characteristics as our target neighborhoods.
----------------------
Bay Ridge
Greenpoint
Crown Heights
Windsor Terrace
Prospect Heights
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Gerritsen Beach
Clinton Hill
Downtown
Boerum Hill
East Williamsburg
North Side
South Side
Fulton Ferry
Vinegar Hill
Dumbo
Bay Ridge
Greenpoint
Crown Heights
Windsor Terrace
Prospect Heights
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Gerritsen Beach
Clinton Hill
Downtown
Boerum Hill
East Williamsburg
North Side
South Side
Fulton Ferry
Vinegar Hill
Dumbo


## Conclusion

As a result of our clustering method, there are **47 neighborhoods** in Brooklyn sharing similar charactersitics as our target neighborhoods, which could help the real estate team narrow down their searches to this neighborhoods for the future investment.